In [28]:
import duckdb
import duckdb
import geopandas as gpd
import pandas as pd
from h3pandas.util.shapely import polyfill
import pyarrow.dataset as ds
from lonboard import viz, Map, SolidPolygonLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.diverging import BrBG_10
from palettable.cartocolors.diverging import Geyser_3, TealRose_7
import numpy as np
import csv

In [29]:
cells = pd.read_csv("cells.csv", header=None)
cells.rename(columns={0: "site", 1: "h3"}, inplace=True)
cells

,site,h3
0,Aldabra Atoll,857b2803fffffff
1,Aldabra Atoll,857b2807fffffff
2,Aldabra Atoll,857b280ffffffff
3,Aldabra Atoll,857b2813fffffff
4,Aldabra Atoll,857b2817fffffff
...,...,...
38536,iSimangaliso Wetland Park,85bcf6cbfffffff
38537,iSimangaliso Wetland Park,85bcf6cffffffff
38538,iSimangaliso Wetland Park,85bcf6d3fffffff
38539,iSimangaliso Wetland Park,85bcf6d7fffffff


In [30]:
con = duckdb.connect()

dataset = ds.dataset("data/h3_5_obisgbif_20240723/", format="parquet")
con.register("dataset", dataset)
con.register("cells", cells)

df = con.execute("""
    select site, species, AphiaID, max(source_gbif) as source_gbif, max(source_obis) as source_obis
    from cells
    inner join dataset on cells.h3 = dataset.cell
    where site = 'Aldabra Atoll' and source_obis is true
    group by site, species, AphiaID
""").fetchdf()

df

,site,species,AphiaID,source_gbif,source_obis,min_year,max_year
0,Aldabra Atoll,Acanthaster planci,213289,False,True,1967.0,1967.0
1,Aldabra Atoll,Zoramia leptacanthus,713323,False,True,NaN,NaN
2,Aldabra Atoll,Vasum turbinellus,211103,True,True,1964.0,1983.0
3,Aldabra Atoll,Umbraculum umbraculum,141879,True,True,1986.0,1986.0
4,Aldabra Atoll,Stylocheilus striatus,224662,True,True,1986.0,1986.0
...,...,...,...,...,...,...,...
1674,Aldabra Atoll,Acropora abrotanoides,207083,False,True,1987.0,2002.0
1675,Aldabra Atoll,Acanthopagrus bifasciatus,218589,True,True,1954.0,1954.0
1676,Aldabra Atoll,Acanthastrea subechinata,288885,False,True,2002.0,2002.0
1677,Aldabra Atoll,Acanthastrea pachysepta,888170,False,True,2002.0,2002.0
